In [ ]:
# processing chain, per ensemble
# 1. basic hourly stats + key radial profiles. Bespoke precip processing (cumulative to hourly)
# 2. interpolate and differencing to get get intermediate steps (offset by half hour)
# 3. linearized intermediate steps, each radial coord now has a row, for convenient filtering
# 4. decay stats...

In [4]:
import os, glob
import wrftools as wrf
import wrfproc as wp
import pandas as pd
import numpy as np
import importlib
importlib.reload(wrf)
importlib.reload(wp)

dfpathroot='dfs'
runpath='../run/'

In [5]:
# set up run list, max run length, and run save name

runpat='run_02_land_t*'
runsname='run_02_land'
nmax=24

# runpat='run_02_CTRL'
# runsname='run_02_CTRL'
# nmax=24*15

runlist=[os.path.basename(p) for p in sorted(glob.glob(os.path.join(runpath,runpat)))]
runlist

['run_02_land_t03',
 'run_02_land_t04',
 'run_02_land_t05',
 'run_02_land_t06',
 'run_02_land_t07',
 'run_02_land_t08',
 'run_02_land_t09',
 'run_02_land_t10',
 'run_02_land_t11',
 'run_02_land_t12',
 'run_02_land_t13',
 'run_02_land_t14',
 'run_02_land_t15']

In [ ]:
# 1.  per ensemble. basic hourly stats + key radial profiles
dfspath=os.path.join(dfpathroot,'df_' + runsname + '.pkl')
dfs=[]
        
for run in runlist:
    print(run)
    wolist=wrf.wolist(run)[:nmax+1]
    df=wp.get_run_max_df(wolist,run,force=False)    
    df=wp.proc_precip(df)    
    dfs.append(df)
    
dfa=pd.concat(dfs).reset_index(drop=True)
sname=os.path.join(dfpathroot,'df_' + runsname + '.pkl')
dfa.to_pickle(dfspath,protocol=4)

run_02_land_t03
wrfout_d03_2007-09-03_00:00:00
wrfout_d03_2007-09-03_01:00:00
wrfout_d03_2007-09-03_02:00:00
wrfout_d03_2007-09-03_03:00:00
wrfout_d03_2007-09-03_04:00:00
wrfout_d03_2007-09-03_05:00:00
wrfout_d03_2007-09-03_06:00:00
wrfout_d03_2007-09-03_07:00:00
wrfout_d03_2007-09-03_08:00:00
wrfout_d03_2007-09-03_09:00:00
wrfout_d03_2007-09-03_10:00:00
wrfout_d03_2007-09-03_11:00:00
wrfout_d03_2007-09-03_12:00:00
wrfout_d03_2007-09-03_13:00:00
wrfout_d03_2007-09-03_14:00:00
wrfout_d03_2007-09-03_15:00:00
wrfout_d03_2007-09-03_16:00:00
wrfout_d03_2007-09-03_17:00:00
Creating: run_02_land_t03/wrfout_d03_2007-09-03_17:00:00.V.az.npy
Creating: run_02_land_t03/wrfout_d03_2007-09-03_17:00:00.V.raw.npy
Creating: run_02_land_t03/wrfout_d03_2007-09-03_17:00:00.vr10.az.npy
wrfout_d03_2007-09-03_18:00:00
Creating: run_02_land_t03/wrfout_d03_2007-09-03_18:00:00.V.az.npy
Creating: run_02_land_t03/wrfout_d03_2007-09-03_18:00:00.V.raw.npy
Creating: run_02_land_t03/wrfout_d03_2007-09-03_18:00:00.vr1

In [17]:
# 2. interpolate and differencing to get get intermediate steps (offset by half hour)
importlib.reload(wp)
dfpath=os.path.join(dfpathroot,'df_' + runsname + '.pkl')
dfspath=os.path.join(dfpathroot,'df_' + runsname + '_delta.pkl')

df=pd.read_pickle(dfpath)
tlfs=df.tlf.unique()
dfl=[]
for tlf in tlfs:
    dft=df[df.tlf.eq(tlf)].reset_index(drop=True) #indexing from 0 is used in create_delta_df
    dfl.append(wp.create_delta_df(dft)[1:]) #first row of diffs is nans
    
dfa=pd.concat(dfl).reset_index(drop=True)
dfa.to_pickle(dfspath,protocol=4)

In [18]:
# 3. tabulated intermediate steps, each radial coord now has a row, 
# for simpler filtering
importlib.reload(wp)
dfpath=os.path.join(dfpathroot,'df_' + runsname + '_delta.pkl')
dfspath=os.path.join(dfpathroot,'df_'+runsname+'_delta_tab.pkl')

df=pd.read_pickle(dfpath)
dfn=wp.tabulate_df_ens([df],200)
dfn.to_pickle(dfspath,protocol=4) 

In [ ]:
dfspath